In [6]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [7]:
# Import tables into dataframes from .txt files
df_outbound = pd.read_csv('Papper_Data/Papper_outbound.txt', sep='\t', index_col=0)
df_inbound = pd.read_csv('Papper_Data/Papper_inbound.txt', sep='\t', index_col=0)
df_CW_costs = pd.read_csv('Papper_Data/Papper_CW_costs.txt', sep='\t', index_col=0)
df_plant_costs_cap = pd.read_csv('Papper_Data/Papper_plant_costs_cap.txt', sep='\t', index_col=0)
df_demand = pd.read_csv('Papper_Data/Papper_demand.txt', sep='\t', index_col=0)

print(df_outbound.describe())


            RW 1       RW 2       RW 3      RW 4       RW 5      RW 6  \
count   5.000000   5.000000   5.000000   5.00000   5.000000   5.00000   
mean   83.600000  89.200000  70.200000  82.20000  82.800000  62.80000   
std     6.913754   8.927486  14.184499  15.12283  12.153189   8.58487   
min    73.000000  78.000000  55.000000  61.00000  62.000000  54.00000   
25%    81.000000  82.000000  63.000000  75.00000  85.000000  58.00000   
50%    85.000000  91.000000  69.000000  82.00000  86.000000  58.00000   
75%    89.000000  97.000000  71.000000  95.00000  87.000000  71.00000   
max    90.000000  98.000000  93.000000  98.00000  94.000000  73.00000   

            RW 7       RW 8       RW 9  
count   5.000000   5.000000   5.000000  
mean   84.400000  81.800000  64.400000  
std    13.427584  12.070626  14.570518  
min    62.000000  72.000000  50.000000  
25%    85.000000  73.000000  55.000000  
50%    88.000000  75.000000  59.000000  
75%    89.000000  90.000000  72.000000  
max    98.0000

In [9]:
# Adjust any columns as necessary (here, RW5 column imported incorrectly, so was renamed)
df_outbound = df_outbound.rename(columns={' RW 5': 'RW 5'})

#Create list of plants, warehouses
cws = df_inbound.index.tolist()
plants = df_inbound.columns.tolist()
rws = df_outbound.columns.tolist()

In [10]:
#create dictionaries for costs and constraints
outbound_dict = df_outbound.to_dict(orient='index')
inbound_dict = df_inbound.to_dict(orient='index')
variable_cost_CW = df_CW_costs['Variable costs'].to_dict()
fixed_cost_CW = df_CW_costs['Fixed costs'].to_dict()
variable_cost_plant = df_plant_costs_cap['Variable costs'].to_dict()
capacity_plant=df_plant_costs_cap['Capacity'].to_dict()
demand_dict = df_demand['Demand'].to_dict()

print(outbound_dict)
print(inbound_dict)
print(variable_cost_CW)
print(fixed_cost_CW)
print(variable_cost_plant)
print(capacity_plant)
print(demand_dict)
print(cws)
print(plants)
print(rws)

{'CW 1': {'RW 1': 89, 'RW 2': 78, 'RW 3': 69, 'RW 4': 95, 'RW 5': 85, 'RW 6': 73, 'RW 7': 88, 'RW 8': 73, 'RW 9': 86}, 'CW 2': {'RW 1': 90, 'RW 2': 91, 'RW 3': 63, 'RW 4': 61, 'RW 5': 94, 'RW 6': 58, 'RW 7': 89, 'RW 8': 90, 'RW 9': 55}, 'CW 3': {'RW 1': 81, 'RW 2': 82, 'RW 3': 71, 'RW 4': 98, 'RW 5': 62, 'RW 6': 71, 'RW 7': 85, 'RW 8': 75, 'RW 9': 72}, 'CW 4': {'RW 1': 73, 'RW 2': 97, 'RW 3': 55, 'RW 4': 75, 'RW 5': 86, 'RW 6': 54, 'RW 7': 62, 'RW 8': 99, 'RW 9': 59}, 'CW 5': {'RW 1': 85, 'RW 2': 98, 'RW 3': 93, 'RW 4': 82, 'RW 5': 87, 'RW 6': 58, 'RW 7': 98, 'RW 8': 72, 'RW 9': 50}}
{'CW 1': {'Plant 1': 77, 'Plant 2': 54}, 'CW 2': {'Plant 1': 48, 'Plant 2': 33}, 'CW 3': {'Plant 1': 41, 'Plant 2': 21}, 'CW 4': {'Plant 1': 57, 'Plant 2': 94}, 'CW 5': {'Plant 1': 94, 'Plant 2': 44}}
{'CW 1': 8, 'CW 2': 10, 'CW 3': 25, 'CW 4': 30, 'CW 5': 50}
{'CW 1': 100000, 'CW 2': 90000, 'CW 3': 80000, 'CW 4': 70000, 'CW 5': 60000}
{'Plant 1': 8, 'Plant 2': 13}
{'Plant 1': 1500, 'Plant 2': 500}
{'RW 1'

In [11]:
# Model
model = gp.Model("Network Design")

# Decision variables
inbound = model.addVars(plants, cws, vtype=GRB.INTEGER, name=f"Inbound")
outbound = model.addVars(cws, rws, vtype=GRB.INTEGER, name=f"Outbound")
fixed = model.addVars(cws, vtype=GRB.BINARY, name=f"Binary")
inbound


Set parameter Username


{('Plant 1', 'CW 1'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 1', 'CW 2'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 1', 'CW 3'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 1', 'CW 4'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 1', 'CW 5'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 2', 'CW 1'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 2', 'CW 2'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 2', 'CW 3'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 2', 'CW 4'): <gurobi.Var *Awaiting Model Update*>,
 ('Plant 2', 'CW 5'): <gurobi.Var *Awaiting Model Update*>}


[cw, rw]: This format is used for Gurobi decision variables (e.g., outbound[cw, rw]), where the variable directly uses both indices in a tuple.

[cw][rw]: This format is used for nested dictionaries (e.g., cw_rw_distances[cw][rw]), where the first key (cw) accesses a dictionary, and the second key (rw) accesses the value within that dictionary.


In [12]:
# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] for plant in plants for cw in cws) + #inbound shipping
    #gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] *0.3 for plant in plants for cw in cws) + #inbound shipping FOR PART 3, REDUCED COST
    gp.quicksum(outbound[cw, rw] * outbound_dict[cw][rw] for cw in cws for rw in rws) +   #outbound shipping
    gp.quicksum(inbound[plant, cw] * variable_cost_plant[plant] for plant in plants for cw in cws) +  #plant variable
    gp.quicksum(outbound[cw, rw] * variable_cost_CW[cw] for cw in cws for rw in rws) +  #CW variable
    gp.quicksum(fixed[cw] * fixed_cost_CW[cw] for cw in cws), #plant fixed
    GRB.MINIMIZE
)
model

<gurobi.Model Continuous instance Network Design: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [13]:
# Set Constraints

# Demand constraint
#REMEMBER TO SET THIS AS == WHEN DEALING WITH LOS CONSTRAINT.
for rw in rws:  
    #model.addConstr(gp.quicksum(outbound[cw, rw] for cw in cws) >= demand_dict[rw], name=f"Demand_Constraint_{rw}")
    model.addConstr(gp.quicksum(outbound[cw, rw] for cw in cws) == demand_dict[rw], name=f"Demand_Constraint_{rw}")  #FOR LOS CONSTRAINT

# Capacity constraint
for plant in plants:  
    model.addConstr(gp.quicksum(inbound[plant, cw] for cw in cws) <= capacity_plant[plant], name=f"Capacity_Constraint_{plant}")

#Conservation of flow constraint
for cw in cws:
    model.addConstr(gp.quicksum(inbound[plant, cw] for plant in plants) == gp.quicksum(outbound[cw, rw] for rw in rws), name=f"Conservation_of_Flow{cw}")

#Linking constraint
M=1000000
for cw in cws:
    model.addConstr(gp.quicksum(outbound[cw, rw] for rw in rws) - fixed[cw] * M <= 0, name=f"Linking_Constraint_{cw}")


#Sum of Y's (open CWs)
y_low = 1
y_lim = 5
model.addConstr(gp.quicksum(fixed[cw] for cw in cws) >= y_low, name="Min_CWs_Open")
model.addConstr(gp.quicksum(fixed[cw] for cw in cws) <= y_lim, name="Max_CWs_Open")

#If Y is open, foce it to be in use
epsilon = 1  # Small value to ensure non-zero flow
for cw in cws:
    model.addConstr(gp.quicksum(outbound[cw, rw] for rw in rws) >= epsilon * fixed[cw], name=f"OutboundActive_{cw}")




In [14]:
#Level of Service Constraints
#MAX DISTANCE FROM DC TO CUSTOMER -----------------------------------------------------------------
def Max_Distance_Condition(value):
    range = 60 #<-------------------- CHANGE THIS AS YOUR RANGE CONDITION
    if value > range:
        return 10000
    else:
        return value
    
# df_Max_Distance = df_outbound.applymap(Max_Distance_Condition)
# df_Max_Distance

#% CUSTOMER WITHIN RANGE OF DC -----------------------------------------------------------------
def Range_Condition(value, max_distance):
    if value > max_distance:
        return 0
    else:
        return 1

# df_Within_Range = df_outbound.applymap(Range_Condition)
# df_Within_Range

def apply_individual_los_constraints(model, df_outbound, outbound, demand_dict, rws, max_distance=70, percentage=0.6):
    # Create df_Within_Range inside this function using the updated max_distance
    df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))

    # Apply the LOS constraint using df_Within_Range
    for rw in rws:
        total_demand_rw = demand_dict[rw]  # Total demand for RW
        required_supply_within_range = percentage * total_demand_rw  # Required supply within range

        # Calculate supply from within range
        supply_from_within_range = gp.quicksum(outbound[cw, rw] * df_Within_Range.loc[cw, rw] for cw in cws)
        
        # Add constraint to the model ensuring at least the required percentage of demand is met from within the specified distance
        model.addConstr(supply_from_within_range >= required_supply_within_range, name=f"LOS_Constraint_{rw}")



#AGGREGATE LOS CONSTRAINT... ALL DEMAND TOTAL HAS TO BE 70% SATISFIED FROM WITHIN 70 MILES ----------------------->
##IMPORTANT#### ------------------------------------------------------> REMEMBE TO CHANGE DEMAND = IN THIS SCENARIO, OR DEMAND MAY BE OVERSUPPLIED. 
def apply_aggregate_los_constraint(model, df_outbound, outbound, demand_dict, cws, rws, max_distance=70, overall_percentage=0.7):
    # Create a DataFrame for within range based on max_distance
    df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))

    # Calculate the total demand across all RWs
    total_demand_all_rws = sum(demand_dict[rw] for rw in rws)

    # Calculate the total required supply within range for all RWs combined
    total_required_supply_within_range = overall_percentage * total_demand_all_rws

    # Sum the supplies from all CWs within 70 miles across all RWs and set the constraint
    model.addConstr(
        gp.quicksum(outbound[cw, rw] * df_Within_Range.loc[cw, rw] for cw in cws for rw in rws) >= total_required_supply_within_range,
        name="Aggregate_LOS_Constraint"
    )


def apply_maximum_distance_constraint(model, outbound, cws, rws, max_distance):
     '''
          Add constraints to the model to ensure deliveries only occur within the maximum allowed distance. 

     '''
     # Create a DataFrame for within range based on max_distance
     df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))
     for rw in rws:
        for cw in cws:
            # Add constraint: outbound[cw, rw] must be zero if the distance is not acceptable
            model.addConstr(outbound[cw, rw] * df_Within_Range.loc[cw, rw] == outbound[cw, rw], name=f"MaxDist_{cw}_{rw}")


#apply_maximum_distance_constraint(model, outbound, cws, rws, max_distance=1000)
apply_aggregate_los_constraint(model, df_outbound, outbound, demand_dict, cws, rws, max_distance=70, overall_percentage=0.6)
#apply_individual_los_constraints(model, df_outbound, outbound, demand_dict, rws, max_distance=80, percentage=0.6)



C:\Users\Local User\AppData\Local\Temp\ipykernel_15024\3350359784.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_Within_Range = df_outbound.applymap(lambda x: Range_Condition(x, max_distance))


In [15]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 60 columns and 232 nonzeros
Model fingerprint: 0x5d01ce29
Variable types: 0 continuous, 60 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [3e+01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 680207.00000
Presolve removed 8 rows and 3 columns
Presolve time: 0.01s
Presolved: 21 rows, 57 columns, 162 nonzeros
Variable types: 0 continuous, 57 integer (2 binary)
Found heuristic solution: objective 680165.00000

Root relaxation: objective 4.908820e+05, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

In [16]:
if model.status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Total Cost: {model.objVal}\n")
    print(f"{'Retail Warehouse':<20} {'Demanded':<10} {'Delivered':<10}")
    print("-" * 40)

    for rw in rws:
        demanded = demand_dict[rw]
        delivered = sum(outbound[cw, rw].X for cw in cws if outbound[cw, rw].X > 0)  # Sum only non-zero deliveries
        print(f"{rw:<20} {demanded:<10} {delivered:<10.2f}")

    print("\nDecision Variable Values (non-zero):")
    print(f"{'Variable':<20} {'Value':<10}")
    print("-" * 30)
    
    for v in model.getVars():
        if v.X != 0:  # Print only non-zero variables
            print(f"{v.varName:<20} {v.X:<10.2f}")

else:
    print(f"Status: {model.status}")



Status: Optimal
Total Cost: 490882.0

Retail Warehouse     Demanded   Delivered 
----------------------------------------
RW 1                 140        140.00    
RW 2                 180        180.00    
RW 3                 240        240.00    
RW 4                 210        210.00    
RW 5                 175        175.00    
RW 6                 130        130.00    
RW 7                 320        320.00    
RW 8                 280        280.00    
RW 9                 160        160.00    

Decision Variable Values (non-zero):
Variable             Value     
------------------------------
Inbound[Plant 1,CW 2] 874.00    
Inbound[Plant 1,CW 3] 275.00    
Inbound[Plant 1,CW 4] 186.00    
Inbound[Plant 2,CW 3] 500.00    
Outbound[CW 2,RW 3]  240.00    
Outbound[CW 2,RW 4]  210.00    
Outbound[CW 2,RW 6]  130.00    
Outbound[CW 2,RW 7]  134.00    
Outbound[CW 2,RW 9]  160.00    
Outbound[CW 3,RW 1]  140.00    
Outbound[CW 3,RW 2]  180.00    
Outbound[CW 3,RW 5]  175.00    
Ou

In [17]:
print('inbound shipping: ', gp.quicksum(inbound[plant, cw] * inbound_dict[cw][plant] for plant in plants for cw in cws).getValue())
print('outbound shipping: ', gp.quicksum(outbound[cw, rw] * outbound_dict[cw][rw] for cw in cws for rw in rws).getValue())
print('plant variable: ', gp.quicksum(inbound[plant, cw] * variable_cost_plant[plant] for plant in plants for cw in cws).getValue())
print('cw variable: ', gp.quicksum(outbound[cw, rw] * variable_cost_CW[cw] for cw in cws for rw in rws).getValue())
print('plant fixed: ', gp.quicksum(fixed[cw] * fixed_cost_CW[cw] for cw in cws).getValue())

inbound shipping:  74329.0
outbound shipping:  125678.0
plant variable:  17180.0
cw variable:  33695.0
plant fixed:  240000.0
